## Define Youtube Class

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from typing import Any, Optional, Dict, List
import json
import datetime
import logging

class CredentialError(Exception):
    """This exception is raised when there is error in creating credential."""

class YoutubeDataError(Exception):
    """This exception is raised when there is error in creating Youtube data object."""

class InsufficientInputError(Exception):
    """This exception is raised when insufficint info is provided to create channel object."""

class ChannelNotFoundError(Exception):
    """This exception is raised when a channel is not found."""

class YoutubeChannel():
    def __init__(self,
                 service_account_info: json,
                 scopes: Optional[list] = ['https://www.googleapis.com/auth/youtube.readonly'],
                 channel_name: Optional[str] = None,
                channel_id: Optional[str] = None) -> None:
        # Trying to create the credential object
        try:
            credentials = service_account.Credentials.from_service_account_info(
            service_account_info,
            scopes=scopes
            )
        except Exception as e:
            raise CredentialError(e)
        
        # Trying to build the youtube object
        try:
            youtube = build('youtube', 'v3', credentials=credentials)
        except Exception as e:
            raise YoutubeDataError(e)
            
        # Fetching Channel id by name
        if channel_name is None and channel_id is None:
            raise InsufficientInputError("Either channel_name or channel_id needs to be provided")
        
        if channel_id is None:
            # Fetch channel ID by name
            response = youtube.search().list(q=channel_name, type='channel', part='id', maxResults=1).execute()
            if not response.get('items'):
                raise ChannelNotFoundError(f"No channel found for username: {channel_name}")
            channel_id = response['items'][0]['id']['channelId']
        else:
            # Verify the channel id passed is correct
            if not youtube.channels().list(id=channel_id,part='id').execute().get('items'):
                raise ChannelNotFoundError(f"No channel found for channel_id: {channel_id}")
        
        # Get channel attributes
        response = youtube.channels().list(
            id=channel_id,
            part='snippet,statistics'
        ).execute()
        
        # Set Channel attribute for the object
        self._credentials = credentials
        self._youtube = youtube
        self.channel_name = channel_name
        self.channel_id = channel_id
        self.title = response.get('items')[0].get('snippet').get('title')
        self.description = response.get('items')[0].get('snippet').get('description')
        self.customUrl = response.get('items')[0].get('snippet').get('customUrl')
        self.publishedAt = response.get('items')[0].get('snippet').get('publishedAt')
        self.country = response.get('items')[0].get('snippet').get('country')
        self.viewCount = response.get('items')[0].get('statistics').get('viewCount')
        self.subscriberCount = response.get('items')[0].get('statistics').get('subscriberCount')
        self.videoCount = response.get('items')[0].get('statistics').get('videoCount')
    
    @staticmethod
    def get_video_statistics(youtube: Any, video_ids: list) -> list:
        video_stats = []

        # Fetch statistics for the videos
        video_response = youtube.videos().list(
            part='snippet,statistics',
            id=','.join(video_ids)
        ).execute()

        for item in video_response['items']:
            video_stats.append({
                'id': item['id'],
                'title': item['snippet']['title'],
                'url': f"https://www.youtube.com/watch?v={item['id']}",
                'views': item['statistics'].get('viewCount',0),
                'likes': item['statistics'].get('likeCount',0),
                'dislikes': item['statistics'].get('dislikeCount',0),
                'comments': item['statistics'].get('commentCount', 0),
                'publishedAt': item['snippet']['publishedAt']
            })
    
        return video_stats

    
    def get_video_data(self, chunk_size: Optional[int] = 50, days_count: Optional[int] = 365):
        video_data = []
        
        # Calculating published_after based on days_count
        t_ago = datetime.datetime.now() - datetime.timedelta(days=days_count)
        published_after = t_ago.isoformat("T") + "Z"

        request = self._youtube.search().list(
                    part='id',
                    channelId=self.channel_id,
                    publishedAfter=published_after,
                    maxResults=chunk_size,
                    type='video'
                )

        while request:
            response = request.execute()
            video_ids = [item['id']['videoId'] for item in response['items']]
            
            # Getting video statistics
            video_data = video_data + self.get_video_statistics(self._youtube, video_ids)
            
            # Creating request for the next chunk fetch
            request = self._youtube.search().list_next(request, response)        
                
        return video_data
        
        

## Define Snowflake Loader class

In [ ]:
import snowflake.connector
from typing import Any, Optional, Dict, List
import logging
        
        
class SnowflakeLoader():
    def __init__(self,
                 conn: Any,
                 schema: str,
                 s3_stage_name: str,
                 stage_table_name: str,
                 core_table_name: str,
                 s3_col_map: Dict,
                 load_type: Optional[str] = 'FULL',
                 merge_on_col: Optional[List[str]] = []
                ):
        try:
            curs = conn.cursor()
        except Exception as e:
            raise Exception(f"Error while opening the cursor. {e}")
        
        # Check when load type is Merge, the merge_on_col need to be supplied
        if load_type.upper() == 'MERGE' and len(merge_on_col) == 0:
            raise Exception(f"merge_on_col arg is mandatory for Loader type: {load_type}")
        
        # Fecthing stage table columns
        curs.execute(f"""
        SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{stage_table_name.upper()}' AND TABLE_SCHEMA = '{schema.upper()}';
        """)
        self.stg_cols = [row[0] for row in curs.fetchall()]
        
        # Fecthing main table columns
        curs.execute(f"""
        SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{core_table_name.upper()}' AND TABLE_SCHEMA = '{schema.upper()}';
        """)
        self.main_cols = [row[0] for row in curs.fetchall()]
        
        # Checking if stage and main table columns are in sync
        if not set(self.main_cols) == set(self.stg_cols):
            raise Exception('Stage and Main table column mismatch. ')
        
        # Converting merge columns to upper
        if  len(merge_on_col) != 0:
            self.merge_on_col = [col.upper() for col in merge_on_col]
        else:
            self.merge_on_col = merge_on_col
            
        # Checking if merge col existing in stage and main table
        if load_type.upper() == 'MERGE':
            if len([col for col in self.merge_on_col if col in self.main_cols and col in self.stg_cols]) != len(self.merge_on_col):
                raise Exception('All columns in merge_on_col must be present in both stage and core table.')
        
            
        self.schema = schema.upper()
        self.stage_table_name = stage_table_name.upper()
        self.core_table_name = core_table_name.upper()
        self.load_type = load_type.upper()
        self.conn = conn
        self.s3_col_map = s3_col_map
        self.s3_stage_name = s3_stage_name
        
        curs.close()
    
    def curs_handler(func):
        def wrapper(self, *args, **kwargs):
            try:
                curs = self.conn.cursor()
            except Exception as e:
                raise Exception(f"from db_conn_check: Error while opening the cursor. {e}")
            func(self, curs, *args, **kwargs)
            curs.close()
        return wrapper

    
    @curs_handler
    def stg_to_core(self, curs, *args, **kwargs) -> None:
            
        if self.load_type == 'MERGE':
            sql_text = [f"""
                MERGE INTO {self.schema}.{self.core_table_name} as t
                USING {self.schema}.{self.stage_table_name} as d
                ON 
                {'AND '.join(f"d.{col} = t.{col} " for col in self.merge_on_col)}
                WHEN MATCHED THEN
                UPDATE SET
                {', '.join(f"{col} = d.{col} " for col in [col for col in self.main_cols if col not in self.merge_on_col])}
                WHEN NOT MATCHED THEN
                INSERT
                ({','.join([col for col in self.main_cols])})
                VALUES
                ({','.join([f"d.{col}" for col in self.main_cols])})
                ;
                """]
        else:
            sql_text = [f"delete from {self.schema}.{self.core_table_name};",
            f"""insert into {self.schema}.{self.core_table_name}
            ({','.join([col for col in self.main_cols])})
            select
            {','.join([col for col in self.main_cols])}
            from
            {self.schema}.{self.stage_table_name};
            """]
        
        for sql in sql_text:
            curs.execute(sql)
            logging.info(sql)

    @curs_handler  
    def s3_to_stg(self, curs, *args, **kwargs) -> None:
        
        # Validate S3_col_map dictionary if available
        for item in self.s3_col_map.items():
            if item[1].upper() not in self.stg_cols:
                raise Exception(f"s3_to_stg: Column {item[1]} not present in the table {self.schema}.{self.stage_table_name}")
                    
        
        # Cleanup stage table first
        curs.execute(f"""delete from {self.schema}.{self.stage_table_name};""")
        
        # Prepare the COPY INTO statement for S3 load
        sql_text = f"""
        COPY INTO {self.schema}.{self.stage_table_name} ({','.join([col for col in [item[1] for item in self.s3_col_map.items()]])})
        FROM (
            SELECT
            {','.join([f"$1:{item[0]}::VARIANT AS {item[1]}" for item in self.s3_col_map.items()])}
            FROM @{self.s3_stage_name}
        )
        FILE_FORMAT = (TYPE = 'PARQUET');
        """
        
        logging.info(sql_text)
        curs.execute(sql_text)
        

## Configuration

In [ ]:
service_account_info = json.load(open('Secrets/youtube-analytics-sph-3bbd40b130f0.json'))
channel_list = ['straitstimesonline', 'BeritaHarianSG1957', 'Tamil_Murasu', 'TheBusinessTimes', 'zaobaodotsg']
temp_directory = './temp_download'
s3_bucket_name = 'youtube-stats-001'
s3_path = "dump/parquet"
aws_access_key_id = "********"
aws_secret_access_key = "************"
sf_username = '**********'
sf_password = '***********'
sf_account = '***********'
sf_warehouse = 'COMPUTE_WH'
sf_database = 'TESTDB'
sf_schema = 'CORE'

## Fetch from API

In [ ]:
import pandas as pd
import datetime
import logging

df_channel = pd.DataFrame(columns=['channel_name','channel_id','title','customUrl','publishedAt','country','viewCount','subscriberCount','videoCount','etl_ts'])
df_video = pd.DataFrame(columns=['id','title','url','views','likes','dislikes','comments','publishedAt','channel_name','channel_id','rptg_dt','etl_ts'])

_now_ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
_today_dt = datetime.datetime.now().strftime("%Y-%m-%d")
logging.info(f"_now_ts: {_now_ts}")
logging.info(f"_today_dt: {_today_dt}")

for channel_name in channel_list:
    logging.info(f"Fetching data for Channel: {channel_name}")
    channelObj = YoutubeChannel(service_account_info=service_account_info, channel_name=channel_name)    
    df_channel = pd.concat([df_channel, pd.DataFrame([{
        'channel_name': channelObj.channel_name,
        'channel_id': channelObj.channel_id,
        'title': channelObj.title,
        'customUrl': channelObj.customUrl,
        'publishedAt': channelObj.publishedAt,
        'country': channelObj.country,
        'viewCount': channelObj.viewCount,
        'subscriberCount': channelObj.subscriberCount,
        'videoCount': channelObj.videoCount,
        'etl_ts': _now_ts
    }])], ignore_index=True)

    
    df_video_temp = pd.DataFrame(channelObj.get_video_data())
    df_video_temp['channel_name'] = channelObj.channel_name
    df_video_temp['channel_id'] = channelObj.channel_id
    df_video_temp['rptg_dt'] = _today_dt
    df_video_temp['etl_ts'] = _now_ts
    df_video = pd.concat([df_video, df_video_temp], ignore_index=True)

df_video_md = df_video[['id','channel_id','title','url','publishedAt','etl_ts']]
df_video = df_video[['id','channel_id','rptg_dt','views','likes','dislikes','comments','etl_ts']]

# Dropping duplicates based on respective Key columns
df_channel = df_channel.drop_duplicates(subset=['channel_id'])
df_video_md = df_video_md.drop_duplicates(subset=['id'])
df_video = df_video.drop_duplicates(subset=['id','rptg_dt'])


In [ ]:
display(df_channel)
display(df_video_md)
display(df_video)

## Write to S3

In [ ]:
import time
import boto3
import io
import pyarrow as pa
import pyarrow.parquet as pq
import logging

# Initialize S3 client
s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Upload the Channel Parquet file to S3
try:
    parquet_buffer = io.BytesIO()
    df_channel.to_parquet(parquet_buffer, index=False)
    chnl_file_name = f"channel_data_{str(int(round(time.time())))}.parquet"
    s3.Object(s3_bucket_name, f"{s3_path}/channel/{chnl_file_name}").put(Body=parquet_buffer.getvalue())
    logging.info(f"File {chnl_file_name} has been uploaded to s3://{s3_bucket_name}/{s3_path}/channel")
except Exception as e:
    raise Exception(f"Channel MD S3 Upload failed. {e}")

# Upload the Video MD Parquet file to S3
try:
    parquet_buffer = io.BytesIO()
    df_video_md.to_parquet(parquet_buffer, index=False)
    video_md_file_name = f"video_md_data_{str(int(round(time.time())))}.parquet"
    s3.Object(s3_bucket_name, f"{s3_path}/video_md/{video_md_file_name}").put(Body=parquet_buffer.getvalue())
    logging.info(f"File {video_md_file_name} has been uploaded to s3://{s3_bucket_name}/{s3_path}/video_md")
except Exception as e:
    raise Exception(f"Video MD S3 Upload failed. {e}")

# Upload the Video Parquet file to S3
try:
    parquet_buffer = io.BytesIO()
    df_video.to_parquet(parquet_buffer, index=False)
    video_file_name = f"video_data_{str(int(round(time.time())))}.parquet"
    s3.Object(s3_bucket_name, f"{s3_path}/video/{video_file_name}").put(Body=parquet_buffer.getvalue())
    logging.info(f"File {video_file_name} has been uploaded to s3://{s3_bucket_name}/{s3_path}/video")
except Exception as e:
    raise Exception(f"Video Stats S3 Upload failed. {e}")


## Load to Snowflake

In [ ]:
import logging

dbcon = snowflake.connector.connect(
    user=sf_username,
    password=sf_password,
    account=sf_account,
    warehouse=sf_warehouse,
    database=sf_database,
    schema=sf_schema
)

logging.info("Start: Loading Channel MD.")
# Load Channel MD
sf_ldr = SnowflakeLoader(
     conn = dbcon,
     schema = sf_schema,
     s3_stage_name = 'stg_yt_channel_md',
     stage_table_name = 'tbl_stg_yt_channel_md',
     core_table_name = 'tbl_yt_channel_md',
     s3_col_map = {
                    'channel_name': 'channel_name',
                    'channel_id': 'channel_id',
                    'title': 'title',
                    'customUrl': 'custom_url',
                    'publishedAt': 'published_at',
                    'country': 'country',
                    'viewCount': 'view_count',
                    'subscriberCount': 'subscriber_count',
                    'videoCount': 'video_count',
                    'etl_ts': 'etl_ts'
                  })

# Run the S3 loader and core loader
sf_ldr.s3_to_stg()
sf_ldr.stg_to_core()
logging.info("Complete: Loading Channel MD.")


logging.info("Start: Loading Video MD.")
# Load Video MD
sf_ldr = SnowflakeLoader(
     conn = dbcon,
     schema = sf_schema,
     s3_stage_name = 'stg_yt_video_md',
     stage_table_name = 'tbl_stg_yt_video_md',
     core_table_name = 'tbl_yt_video_md',
     s3_col_map = {
                    'id': 'id',
                    'channel_id': 'channel_id',
                    'title': 'title',
                    'url': 'url',
                    'publishedAt': 'published_at',
                    'etl_ts': 'etl_ts'
                  },
     load_type = 'MERGE',
     merge_on_col = ['id']
        )

# Run the S3 loader and core loader
sf_ldr.s3_to_stg()
sf_ldr.stg_to_core()
logging.info("Complete: Loading Video MD.")


logging.info("Start: Loading Video Stats.")
# Load Video stats
sf_ldr = SnowflakeLoader(
     conn = dbcon,
     schema = sf_schema,
     s3_stage_name = 'stg_yt_video_stats',
     stage_table_name = 'tbl_stg_yt_video_stats',
     core_table_name = 'tbl_yt_video_stats',
     s3_col_map = {
                    'id': 'id',
                    'channel_id': 'channel_id',
                    'rptg_dt': 'rptg_dt',
                    'views': 'view_count',
                    'likes': 'like_count',
                    'dislikes': 'dislike_count',
                    'comments': 'comment_count',
                    'etl_ts': 'etl_ts'
                  },
     load_type = 'MERGE',
     merge_on_col = ['id','rptg_dt']
)

# Run the S3 loader and core loader
sf_ldr.s3_to_stg()
sf_ldr.stg_to_core()
logging.info("Complete: Loading Video Stats.")
